# Demo  

This is a demonstration notebook of how fast annealed Langevin vs. RMSGLD converges for the NCSN model.

In [9]:
import os
import shutil
import yaml
import sys
import torch
import argparse
import numpy as np
from runners import *
from main import dict2namespace

We load the weights from the original NCSN paper, which uses `yaml` file to configure the model. The configuration file is `configs/anneal.yml`.

In [24]:
with open(os.path.join('configs', 'anneal.yml'), 'r') as f:
    config = yaml.safe_load(f)
config['device'] = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
config = dict2namespace(config)

Addtionally, the `args` must be passed into the runner to get directories for image and log outputs.

In [25]:
args = argparse.Namespace()
setattr(args, 'test', False)
setattr(args, 'log', 'run/logs/mnist/')
setattr(args, 'image_folder', 'images/mnist/')

In [26]:
runner = AnnealRunner(args, config)

In [ ]:
runner.test()